In [8]:
from glob import glob
from os.path import join, basename

import pandas as pd
import numpy as np
from biosppy.signals.tools import welch_spectrum
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
common_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs"
#common_path = "/Volumes/MMIS-Saraiv/Datasets/Sapienza/denoised_txt_epochs_matlab"
out_path = '/Volumes/MMIS-Saraiv/Datasets/Istambul/iaf_tf_my-boudaries.csv'
sf = 128.0

In [10]:
all_subjects = glob(join(common_path, '*'))
all_subjects

['/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/alzhay1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/alzhpeh1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/alzmsap1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/d1kyil1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eoabob1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eocars1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eoeakt1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eofyil1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eohboz1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eoherk1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eohuzu1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eoibal1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eomrus1',
 '/Volumes/MMIS-Saraiv/Datasets/Istambul/denoised_txt_epochs/eomtyil1',
 '/

In [11]:
res = pd.DataFrame(columns=['Subject', 'IAF', 'TF'])

In [12]:
MULTIPLIER = 2
for subject_path in all_subjects:
    subject = basename(subject_path)
    #subject = basename(subject_path).split('PARTICIPANT')[1]
    print("Subject Code", subject)
    all_files = glob(join(subject_path, '*.txt'))
    power = []
    freqs = None
    # go by epoch
    for file in all_files:
        data = pd.read_csv(file, sep='\t', dtype=float)  # time x channels
        # select electrodes O1 (18), Oz (19), O2 (20)
        freqs1, power1 = welch_spectrum(data.iloc[:, 17], sampling_rate=sf, decibel=True, size=int(sf*MULTIPLIER-1))
        freqs2, power2 = welch_spectrum(data.iloc[:, 18], sampling_rate=sf, decibel=True, size=int(sf*MULTIPLIER-1))
        freqs3, power3 = welch_spectrum(data.iloc[:, 19], sampling_rate=sf, decibel=True, size=int(sf*MULTIPLIER-1))
        assert np.all(freqs1 == freqs2) and np.all(freqs2 == freqs3)
        # Average power of the three electrodes
        power.append((power1 + power2 + power3) / 3)
        freqs = freqs1
    
    # average power across epochs
    power_avg = np.mean(np.array(power), axis=0)
    
    # find IAF between 8 and 14 Hz
    iaf = freqs[np.argmax(power_avg[8*MULTIPLIER:12*MULTIPLIER]) + 8*MULTIPLIER]
    iaf = round(iaf, 1)
    print("IAF", iaf)
    # find TF between 3 and 8 Hz
    tf = freqs[np.argmin(power_avg[4*MULTIPLIER:8*MULTIPLIER]) + 4*MULTIPLIER]
    tf = round(tf, 1)
    print("TF", tf)
    
    # plot all psds
    """
    sns.set_palette("husl")
    for p in power:
        sns.lineplot(x=freqs[:46], y=p[:46], alpha=0.05, color='orange')
    sns.lineplot(x=freqs[:46], y=power_avg[:46], linewidth=3, color='black')
    plt.axvline(iaf, color='red', linestyle='--')
    plt.axvline(tf, color='blue', linestyle='--')
    sns.despine()
    plt.title("Subject Code {}".format(subject))
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power (dB)")
    plt.show()
    """
    
    res = res.append({'Subject': subject, 'IAF': iaf, 'TF': tf}, ignore_index=True)

Subject Code alzhay1
IAF 9.0
TF 5.5


/var/folders/1g/sc9mrb8x18s7r9d97p9glqch0000gp/T/ipykernel_57418/1816990409.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Subject': subject, 'IAF': iaf, 'TF': tf}, ignore_index=True)


Subject Code alzhpeh1
IAF 8.0
TF 4.0
Subject Code alzmsap1


/var/folders/1g/sc9mrb8x18s7r9d97p9glqch0000gp/T/ipykernel_57418/1816990409.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Subject': subject, 'IAF': iaf, 'TF': tf}, ignore_index=True)


IAF 8.0
TF 7.5
Subject Code d1kyil1


...

/var/folders/1g/sc9mrb8x18s7r9d97p9glqch0000gp/T/ipykernel_57418/1816990409.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Subject': subject, 'IAF': iaf, 'TF': tf}, ignore_index=True)


IAF 10.0
TF 6.5
Subject Code pukab1


/var/folders/1g/sc9mrb8x18s7r9d97p9glqch0000gp/T/ipykernel_57418/1816990409.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Subject': subject, 'IAF': iaf, 'TF': tf}, ignore_index=True)


IAF 8.0
TF 5.5


/var/folders/1g/sc9mrb8x18s7r9d97p9glqch0000gp/T/ipykernel_57418/1816990409.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Subject': subject, 'IAF': iaf, 'TF': tf}, ignore_index=True)


In [13]:
res.index = res['Subject']
res = res.drop(columns=['Subject'])
res

,IAF,TF
Subject,,
alzhay1,9.0,5.5
alzhpeh1,8.0,4.0
alzmsap1,8.0,7.5
d1kyil1,9.5,7.5
eoabob1,10.0,5.5
...,...,...
pspam1,9.0,4.5
pssen1,8.0,4.5
pssog1,9.5,5.5


In [14]:
# Save to CSV
res.to_csv(out_path)